<a href="https://colab.research.google.com/github/PUC-RecSys-Class/RecSysPUC-2020/blob/master/practicos/pyRecLab_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Ignacio Contreras Mc Kay

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**
Es de filtrado colaborativo debido a que utiliza los ratings generados por los usuarios que han visto películas semejantes al usuario activo. Este método está basado en items ya que busca la diferencia entre los ratings promedios entre los items y los utiliza como "pendiente" sobre el rating del usuario activo en el item i para obtener el del item j.
Este algoritmo recibe los ratings de los usuarios y obtiene, como dije, el promedio de los ratings por item. Luego, se utilizan estos promedios para obtener la diferencia entre los items que el usuario ha visto y el item a predecir y ponderar, con las similitudes entre i y j, el rating esperado del item a predecir.
Si se agrega un nuevo rating se computa nuevamente la diferencia o pendiente entre los items i y j. Para todas los items j que pertenecen al conjunto de items "vistos" por usuarios que han visto ambos items.



# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [ ]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    635      0 --:--:-- --:--:-- --:--:--   633
100 1546k  100 1546k    0     0  1868k      0 --:--:-- --:--:-- --:--:-- 1868k


In [ ]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1089      0 --:--:-- --:--:-- --:--:--  1092
100  385k  100  385k    0     0   672k      0 --:--:-- --:--:-- --:--:--  672k


In [ ]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1492      0 --:--:-- --:--:-- --:--:--  1492
100  230k  100  230k    0     0   503k      0 --:--:-- --:--:-- --:--:--  503k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [ ]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 194kB 2.8MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [ ]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [ ]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [ ]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [ ]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**
Calcula los slopes promedios para cada par de items haciendo referencia a los usuarios que hicieron rating en esos items i, j. Esto para cuando se pida la predicción para un item especifico para el usuario activo hacer f(x) = x + b, con b el slope y obtener el rating.

## Paso 6:

Llego la hora de predecir el rating.

In [ ]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [ ]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [ ]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [ ]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** Esto se explica debido a que se consideran los promedios de ratings de los items y no por valores individuales. Por lo que, la diferencia a añadir al rating del item i para el nuevo item 9, no va a generar un 5 exacto. Sino que se acercará y hasta podría parecerse más a 4 si es que el promedio del item 9 es muy bajo.

In [ ]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [ ]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [ ]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Elegí el usuario 200 y me entregó 4 de las 5 películas recomendadas al usuario 457.

In [ ]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("200", 10)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1653    Entertaining Angels: The Dorothy Day Story (1996)
1064                                     Crossfire (1947)
1651                         Spanish Prisoner, The (1997)
1650                              Butcher Boy, The (1998)
1645                              Butcher Boy, The (1998)
1643                                    Angel Baby (1995)
1642                             Some Mother's Son (1996)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** Se utilizó el algoritmo Slope One y Most popular para el usuario 44 y se entregaron las top 10 películas recomendadas en base al rating esperado. Observando los resultados, nos damos cuenta de que son idénticos, para top_n igual a 5, 10, 20. Esto se debe a que 

In [ ]:
# Slope One
reclist_slopeone = mySlopeOne.recommend("44", 20)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
1463                          Boys, Les (1997)
1449                    Pather Panchali (1955)
1367                              Faust (1994)
1293                           Star Kid (1997)
868                    Hearts and Minds (1996)
814              Great Day in Harlem, A (1994)
150                            Swingers (1996)
149                                Jude (1996)
145                  Lawnmower Man, The (1992)
143                 Sound of Music, The (1965)
Name: title, dtype: object

In [ ]:
# Most popular
most_popular = pyreclab.MostPopular(dataset='u1.base',
                dlmchar=b'\t',
                header=False,
                usercol=0,
                itemcol=1,
                ratingcol=2)

# en teoria no es "entrenar" , objeto "most_popular" solo registra los items más populares en el set de entrenamiento  
most_popular.train()

In [ ]:
ranking = [int(r) for r in most_popular.recommend("44", 20, includeRated=False)]
print('Recommendation for user {}: {}'.format("44", ranking))
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Recommendation for user 44: [50, 100, 181, 286, 1, 121, 300, 127, 7, 98, 172, 56, 237, 117, 222, 204, 313, 173, 79, 151]
Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
1463                          Boys, Les (1997)
1449                    Pather Panchali (1955)
1367                              Faust (1994)
1293                           Star Kid (1997)
868                    Hearts and Minds (1996)
814              Great Day in Harlem, A (1994)
150                            Swingers (1996)
149                                Jude (1996)
145                  Lawnmower Man, The (1992)
143                 Sound of Music, The (1965)
Name: title, dtype: object